# GSM-VE: Visco-elastic model

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
from bmcs_matmod.api import GSM1D_VE
gsm = GSM1D_VE()
display(Markdown(gsm.latex_potentials()))

In [ ]:
E_val = 30e+3  # Young's modulus in MPa
eta_val = 30e+4  # Viscosity modulus in Pa / s
mparams = dict(E=E_val, eta_ve=eta_val)
d_t_val = 0.01 # Time step in seconds
final_time = 20  # Final time in secondsfinal_time = 1

# Generate cyclic sinusoidal loading history
# time_t = np.arange(0, final_time, d_t_val)
# eps_t_2_cycles = 0.01 * np.sin(2 * np.pi * 1 * time_t / final_time)  # 2 cycles
# eps_t_4_cycles = 0.01 * np.sin(2 * np.pi * 4 * time_t / final_time)  # 4 cycles
time_t = np.linspace(0, final_time, 10)
eps_t_2_cycles = 2 * time_t

In [ ]:
fig, ax = plt.subplots(1, 1)
n_I = 1
t_t = time_t
eps_n_t = eps_t_2_cycles
t_t, eps_ta_nr_coarse, sig_tIa, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm.get_response(eps_n_t, t_t, **mparams)
ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='blue')
# eps_n_t = eps_t_4_cycles
# t_t, eps_ta_nr_coarse, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_ve.get_response(eps_n_t, t_t, 100, *mparams)
# ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='red')

In [ ]:
mparams = dict(E = 20, K = 10, S = 10, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=30, eta_ve=10)

n_t = 151
n_I = 1
sig_max = 10
final_time = 10
t_t = np.linspace(0, final_time, n_t)
sig_n_t = np.linspace(0, sig_max, n_t)
response = gsm.get_G_response(sig_n_t, t_t, **mparams)


In [ ]:
fig, ax_sig = plt.subplots(1, 1, figsize=(8, 5))

ax_iter = ax_sig.twinx()

t_t, sig_ta, eps_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
print(iter_t.shape)
eps_t = eps_ta[:, 0]
iter_t = iter_t[:, 0]
sig_t = sig_ta[:, 0]
ax_sig.plot(eps_t, sig_t)
ax_iter.plot(eps_t, iter_t, ls='dashed')
# Add a box with the material parameters
param_text = ', '.join([f'{key}: {value}' for key, value in mparams.items()])
param_text = '\n'.join([', '.join(param_text.split(', ')[i:i+5]) for i in range(0, len(param_text.split(', ')), 5)])
props = dict(boxstyle='round', facecolor='lightgray', edgecolor='lightgray', alpha=0.5)
ax_sig.text(0.5, 0.97, param_text, transform=ax_sig.transAxes, fontsize=11,
            verticalalignment='top', horizontalalignment='center', bbox=props)
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')